# Les interruptions

Les interruptions sont un mécanisme pour réagir à la survenue d'événement. Dans un système de controle industriel, ou de management d'un moteur, un événement peut survenir d'un capteur physique suite au changement de pression ou de température par exemple. Les événements peuvent aussi être engendré à l'intérieur d'un ordinateur: par l'horloge matérielle, un appui sur la touche d'un clavier ou un déplacement de la souris.

Sans les interruptions, le seul moyen de gérer de tels événements est de «sonder» toutes les sources possibles de tels événement, régulièrement, afin de voir s'il y a eu une activation. Cette façon de procéder n'est pas efficace et requiert beaucoup de code spécialisé. Il est, de plus, difficile de s'assurer que le «sondage» est effectué de façon régulière, car la réponse à un événement donné peut prendre un temps variable. Et, dans la plupart des cas, il y a un «programme principal» qui doit être exécuté pendant ce temps là.

Une interruption est un mécanisme matériel qui a pour effet de mettre en pause l'exécution du programme courant, d'effectuer un branchement vers un type particulier de sous-routine qui détermine la source et la nature de l'événement, puis effectue immédiatement toute action appropriée, puis enfin reprend l'éxécution du programme courant au point où il a été interrompu.

## Encart - les interruptions et le cycle chargement-décodage-exécution (fetch-decode-execute cycle)

Les interruptions sont seulement gérer à la fin de chaque cycle «chargement-décodage-exécution». Pourquoi? Permettre les interruptions durant ce cycle empêcherait de sauvegarder l'état du processeur (de ses registres) de manière consistente, et donc il ne serait pas possible de reprendre le programme interrompu, de manière sûre, une fois l'interruption traitée.

## Encart - Interruptions en cascade

Sur les systèmes qui ont de nombreuses interruptions, il y a un risque d'interruptions en cascade (*interrupt storm* - «orage d'interruptions»): de nouvelles interruptions se produisent à un rythme tel qu'il ne soit pas possible de les traiter. Les concepteurs de système informatique doivent éviter cette situation, par exemple:
- en s'assurant que la vitesse d'exécution rendent la chose hautement improbable,
- en permettant d'associer des niveaux de priorités matériel aux diverses interruptions de façon qu'une interromption de plus haute priorité puisse interrompre une interruption de priorité plus faible,
- en régulant délibéremment leur débit: en utilisant le matériel pour limiter l'intervalle de temps minimum entre deux interruptions successives.

## broche d'interruption

Les premiers microprocesseurs était muni d'une broche étiqueté avec quelquechose comme *IRQ* (pour *Interrupt ReQuest*). L'ARMlite simule un telle broche, mais elle n'est visible à l'écran qu'après avoir activé \[ *enable* \] explicitement les interruptions. Nous allons les utiliser à partir de maintenant. Premièrement, nous avons besoin d'un programme principale pour occuper le processeur:

```
// Programme principal
    MOV R1, #.PixelScreen
    MOV R2, #0 //index du pixel courant
boucle: LDR R0, .Random // couleur au hasard
    STR R0, [R1+R2]
    ADD R2, R2, #4
    CMP R2, #12288 //64x32 (mid-res)
    BLT boucle
    MOV R2, #0
    B boucle
```

#### Exercice 34

Faites fonctionner ce programme et vérifier par vous-même qu'il tourne vite et sans s'interrrompre. Mettez le en pause à un moment et faites une copie d'écran de sa zone d'affichage.
____

Nous voudrions interrompre cette routine principal \[ *main* \], et écrire quelquechose dans la console à chaque fois. Pour faire en sorte que les choses restent simples, nous écrirons une sous-routine qui écris juste le caractère A dans la console à chaque fois qu'elle est appelée:

```
// Routine d'interruption
ecrireA: PUSH {R0}
    MOV R0, #65
    STR R0, .WriteChar
    POP {R0}
    RFE
```

*Notes*:
- Comme pour toute sous-routine, si nous utilisons un (ou des) registre(s), nous devons prendre garde à sauvegarder/restaurer leur contenu en utilisant la pile système avec `PUSH` et `POP`,
- Cependant, pour une routine d'interruption, la sauvegarde/restauration de la valeur du registre `LR` pour retrouver le point de retour, n'est pas nécessaire. En effet, lorsqu'une routine d'interruption est appelée, le processeur n'utilise pas la valeur de `LR` (nous verrons comment il procède un peu plus loin).
- En raison de cette approche particulière du retour au programme principale, nous terminons notre routine d'interruption avec une instruction `RFE` (pour *Return From Exception*), «exception» pour marquer le caractère exceptionnel d'une interruption.

Ayant introduit nous routine d'interruption, nous devons préciser où et quand elle doit être appelée. C'est ce que réalise le code qui suit, au *départ du programme*:

```
// Activation des interruptions
    MOV R0, #ecrireA
    STR R0, .PinISR  // précise l'adresse de démarrage de la routine lorsque la broche d'interruption est activée
    MOV R0, #1
    STR R0, .PinMask // Active la broche d'interruption en la positionnant à 1 (plutôt que 0 par défaut)
    STR R0, .InterruptRegister // Active les interruptions en général
```

*Notes*:
- Comme détaillé dans les commentaires, ce code réalise trois opérations nécessaire au bon fonctionnement des interruptions,
- Il peut sembler étrange qu'après avoir préciser que la broche d'interruption (si activé) doit appeler la routine `ecrireA`, il faille encore activer la broche d'interruption et encore les interruptions en général. C'est nécessaire car (comme nous le verrons) l'ARMlite gère plusieurs sortes d'interruptions, et, pour certaines applications, il est important de pouvoir activer/désactiver \[ *enable/disable* \] chaque sorte d'interruption, voir les désactiver toutes, durant certaine parties critiques du programme principal.
- la forme d'écriture de l'ARMLite à propos des interruptions est très proche de celle d'un processeur ARM réel, bien que les sortes d'interruptions géré par ce simulateur lui soit particulières.

#### Exercice 35

Ajouter le code de la routine d'interruption après celui du programme principale puis le code d'activation des interruptions au début.

Faites tourner le programme complet et vérifier que le programme principal continu à s'exécuter comme avant.

Vous voyez à présent que la broche d'interruption est apparue dans la zone Processor:

Vérifier qu'à chaque fois que vous cliquez sur cette icone (le petit éclair), un A est écris dans la console.

Est-ce que cette interruption semble ralentir le programme principal?

Mettre le programme en pause, poser un point d'arrêt sur l'intruction `MOV R0, #65` puis relancer l'exécution.

Cliquer sur la broche d'interruption (souvenez-vous que le processeur se met en pause juste avant que le point d'arrêt ne soit atteint).

Quelle est la valeur du registre de lien LR?

*Sans relancer l'exécution*, afficher la page mémoire ffe de façon à voir la pile à la fin de la page. Le pointeur de pile SP vous indique l'adresse mémoire du sommet de la pile à ce point. Cela vous permet de constater qu'il y a trois valeurs dans la pile à cet instant. Faites une copie d'écran qui montre ces trois valeurs.

Quelle est la signification de la valeur au sommet de la pile (c'est-à-dire à l'adresse mémoire contenue dans SP)? (*indice*: rechercher cette valeur ailleurs sur l'écran)

Appuyer sur Single Step quatre fois. À partir de quelle instruction le programme principal reprend-t-il son exécution? Comment le processeur sait à quelle instruction il doit reprendre son exécution?
____

La dernière valeur dans la pile (0x80000001) est le moyen par lequel le processeur sauvegarde son état courant qui inclu notamment les quatre bits de status (*Status bits*) que vous pouvez voir dans l'UI de l'ARMLite. C'est nécessaire car cet état pourrait être modifié à l'interieur de la routine d'interruption, il faut donc être en mesure de le restaurer.

L'icone de broche d'interruption simule la façon dont une broche d'interruption matérielle fonctionne sur un microprocesseur réel. Sur un véritable ordinateur, cette broche serait connectée à un ou plusieurs périphérique \[ *device* \] matériel comme le clavier, la souris, ou une horloge matérielle (qui produit des impulsions régulières). L'ARMlite simule de telles possibilités.

## Interruptions clavier - *keyboard interrupts*

En modifiant légèrement le code précédent, comme indiqué ci-dessous, nous pouvons utiliser la simulation des interruptions clavier.

```
// Activer les interruptions
    MOV R0, #ecrire            //plutôt que #ecrireA
    STR R0, .KeyboardISR       //précédemment .PinISR (ISR pour Interrupt Service Routine)
    MOV R0, #1
    STR R0, .KeyboardMask      //précédemment .pinMask
    STR R0, .InterruptRegister
    
// Programme principal
    MOV R1, #.PixelScreen
    MOV R2, #0 //index du pixel courant
boucle: LDR R0, .Random // couleur au hasard
    STR R0, [R1+R2]
    ADD R2, R2, #4
    CMP R2, #12288 //64x32 (mid-res)
    BLT boucle
    MOV R2, #0
    B boucle

// routine d'interruption
ecrire: PUSH {R0}
    LDR R0, .LastKey //précédemment #65 pour A
    STR R0, .WriteChar
    POP {R0}
    RFE
```

#### Exercice 36

Faites tourner ce programme. Vois-t-on la proche d'interruption?

Pendant que le programme tourne, taper des caractères sur le clavier et vérifier qu'ils sont écris immédiatement dans la console.

Fait-il la différence entre des lettres en majuscules ou en minuscules?
__________

## Interruptions d'horloge - *clock interrupts*

Beaucoup d'application ont besoin d'avoir une connaissance précise de l'écoulement du temps, pour rythmer les choses. Elles accomplissent cela par le moyen d'une horloge matérielle qui interrompt le processeur à un rythme régulier. L'ARMlite simule une telle horloge comme vous le montre le code suivant dont l'effet est de faire clignoter un large pixel entre noir et blanc.

```
// Activer les interruptions
    MOV R0, #pixelHorloge
    STR R0, .ClockISR
    MOV R0, #1000
    STR R0, .ClockInterruptFrequency
    MOV R0, #1
    STR R0, .InterruptRegister

// programme principal
main: B main // juste une boucle vide

pixelHorloge:
    PUSH {R0, R1}
    MOV R1, #.white
    LDR R0, .Pixel0
    EOR R0, R0, R1
    STR R0, .Pixel0
    POP {R0, R1}
    RFE
```

#### Exercice 37

Faites tourner ce programme et mesurer le temps de clignotement avec votre montre - Quelle est sa périodicité?

La fréquence des interruptions peut être réglée par **ClockInterruptFrequency**.

Comment procéderiez-vous pour que la vitesse de clignottement soit d'un battement par seconde?
____

## Interruptions lors d'un clic sur un pixel

L'autre forme d'interruptions que l'ARMlite sait gérer est le clic avec la souris sur un pixel de la zone d'affichage (en moyenne ou haute résolution). Cela peut servir pour écrire une simple application de dessin ou pour des jeux interactifs.

L'exemple qui suit peint un pixel en rouge à l'endroit où l'utilisateur clique sur la zone d'affichage.

```
// Activer les interruptions
    MOV R0, #peindre
    STR R0, .PixelISR
    MOV R0, #1
    STR R0, .PixelMask
    STR R0, .InterruptRegister

// initialiser la couleur
    MOV R12, #.red
main: B main

// Routine d'interruption pour peindre un pixel cliqué par l'utilisateur
// La couleur est précisée dans R12 (variable global)
peindre: PUSH {R1, R2}
    MOV R1, #.PixelScreen
    LDR R2, .LastPixelClicked
    LSL R2, R2, #2 // multiplier ce nombre par 4 pour obtenir l'adresse du mot correspondant
    STR R12, [R1+R2]
    POP {R1, R2}
    RFE
```

#### Exercice 38

Faites tourner ce programme et vérifier que vous pouvez cliquer sur un pixel arbitraire qui sera peint en rouge.

Créer une routine d'interruption supplémentaire qui est pilotée par les interruptions clavier (se référer à la section précédente) et, qui, selon la touche enfoncée parmi deux (ou plus) possibles, change la couleur dans la variable globale R12.

Faites tourner votre programme et montrer, avec une capture d'écran que vous pouvez à présent peindre des pixels avec différentes couleurs (au moins 2). Montrer aussi votre code.
__________

### Encart - Les interruptions et le système d'exploitation ??

Comme vous l'avez peut-être compris, le code que vous avez écris pour lire le clavier et écrire des caractères dans la console, pour utiliser l'horloge et pour répondre aux clics de la souris, pendant que d'autres processus continuent à tourner, émule, de façon simple, deux des principales fonctions du système d'exploitation de votre ordinateur: la gestion des interruptions et l'ordonnancement des processus (quel processus s'exécute à quel moment).